Применение и последующее сравнение различных методов машинного обучения в задаче предсказания температуры плавления халькогенидных стекол в зависимости от их элементного состава

In [27]:
# Объявляю нужные библиотеки
import pandas as pd #Библиотека для работы с фреймворками
import numpy as np #Библиотека для математических операций

Работа с исходными данными, взятыми из научной статьи *S. Singla et al. (2023)* [<u>https://doi.org/10.1088/2515-7639/acc6f2</u> ; <u>https://github.com/M3RG-IITD/chgs-ai</u>]. В качестве объекта выступает температура плавления халькогенидных стекол. Признаками же является молярное содержание тех или иных атомов халькогенов.

In [ ]:
# Загрузка датасета из csv-файла
df = pd.read_csv('TG.csv', sep=',')
# Выделение из датасета объектов и признаков
X_data = df.iloc[:,:-1].to_numpy() # Выделение признаков в отдельный массив
Y_data = df['TG'].to_numpy() # Выделение объектов в отдельный массив